In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from scipy import integrate 
from scipy import optimize
from scipy.interpolate import interp1d
from datetime import datetime
from metpy.units import units
from siphon.simplewebservice.wyoming import WyomingUpperAir

ModuleNotFoundError: No module named 'xarray'

# Read data

In [ ]:
%run profile_read_py.py

In [ ]:
arquivo = '/Volumes/GoogleDrive/My Drive/Estudos/IC/Henrique/Programs/RMlicelPython/RM1183023.324' #30 agosto 2011

read = profile_read(arquivo)
phy  = read[1]

df = pd.DataFrame(phy)

def SetHeader(df):
    # Aqui é um jeito provisório de fazer o header

    lista = []
    for i in range(len(read[0]['ch']['active'])):
        wlen = str(read[0]['ch']['wlen'][i])
        pol = str(read[0]['ch']['pol'][i])
        photon = read[0]['ch']['photons'][i]
        an = 'analog' if photon == 0 else 'photon'
        tr = read[0]['ch']['tr'][i]
        lista.append(wlen + ' ' + pol + ' ' + an + ' ' + tr[-1])

    ###############################################


    df.columns = lista
    alt = 7.5*(np.array(df.index) + 1)

    df['alt'] = alt
    s355 = df['355 o analog 0']

    df.set_index('alt', inplace=True)
    
    return

SetHeader(df)

In [ ]:
plot = df[:3000].plot(grid=True, subplots=True, title='First test plot')

# Reading multiple data (mean signal)

The idea here is to iterate over the data files in a month directory.

In [ ]:
import os

directory = '/Volumes/GoogleDrive/My Drive/Estudos/IC/Henrique/Programs/RMlicelPython/30'

initial = 0
final = 0 + 30             # Tomando medias de 30 minutos

data = np.zeros((4000, final - initial))  # Sao sempre 4000 dados (?)
cont = 0

for filename in os.listdir(directory)[initial:final]:
    read = profile_read(directory + '/' + filename)
    data[:, cont] = read[1][:, 0]
    phy = read[1]
    cont += 1
    #print(data)

In [ ]:
import os

directory = '/Volumes/GoogleDrive/My Drive/Estudos/IC/Henrique/Programs/RMlicelPython/30'

initial = 0
final = 0 + 30             # Tomando medias de 30 minutos

data = np.zeros((4000, final - initial))  # Sao sempre 4000 dados (?)
cont = 0

# Lendo o primeiro arquivo pra depois concatenar os proximos

filename = os.listdir(directory)[initial]
read     = profile_read(directory + '/' + filename)
df1      = pd.DataFrame(read[1])
df3      = df1 # TESTE
for filename in os.listdir(directory)[initial+1:final]:
    read = profile_read(directory + '/' + filename)
    df2  = pd.DataFrame(read[1])
    df1  = pd.concat((df1, df2))

df1 = df1.groupby(level=0).mean()
df1

In [ ]:
data

In [ ]:
os.listdir(directory)[initial]

In [ ]:
# Aqui é um jeito provisório de fazer o header

lista = []
for i in range(len(read[0]['ch']['active'])):
    wlen   = str(read[0]['ch']['wlen'][i])
    pol    = str(read[0]['ch']['pol'][i])
    photon = read[0]['ch']['photons'][i]
    an     = 'analog' if photon == 0 else 'photon'
    tr     = read[0]['ch']['tr'][i]
    lista.append(wlen + ' ' + pol + ' ' + an + ' ' + tr[-1])

###############################################


df.columns = lista
alt = 7.5*(np.array(df.index) + 1)

df['alt'] = alt
s355 = df['355 o analog 0']

df.set_index('alt', inplace=True)

df.head()

In [ ]:
plot = df[:3000].plot(grid=True, subplots=True, title= f'Mean of signals between {initial} and {final} minutes')

## Para conferir se os dados estão sendo lidos

In [ ]:
#os.listdir(directory)

In [ ]:
df = pd.DataFrame(data)
#df.columns = lista
df

In [ ]:
s355 = df.mean(axis=1)
s355

In [ ]:
import numpy as np
import pandas as pd

# some random data frames
df1 = pd.DataFrame(dict(x=np.random.randn(100), y=np.random.randint(0, 5, 100)))
df2 = pd.DataFrame(dict(x=np.random.randn(100), y=np.random.randint(0, 5, 100)))

# concatenate them
df_concat = pd.concat((df1, df2))

print(df_concat.mean())
# x   -0.163044
# y    2.120000
# dtype: float64

print(df_concat.median())
# x   -0.192037
# y    2.000000
# dtype: float64

# Applying corrections to signal data

In [ ]:
plt.plot(s355, alt*1e-3)
plt.grid(True)
plt.ylabel('Altitude (km)')
plt.xlabel('Signal [a. u.]')
plt.ylim(0, 4)
plt.savefig('fig1', dpi=200)

## Picture for $log(P r^2)$

In [ ]:
plt.plot(np.log(s355*alt*alt), alt*1e-3)
plt.grid(True)
plt.legend(['355 nm'])
plt.ylabel('Altitude (km)')
plt.xlabel('Log Range Corrected Signal [a. u.]')
plt.ylim(0, 15)
plt.savefig('fig3', dpi=200)

# Read temperature and pressure data

## Usando perfil artificial

z = alt
Pair = read[0]['P0']*np.exp(-(z-100)/8e3)    # H aproximadamente 8km
Tair = read[0]['T0'] - (9.8*1e-3)*z    # 6.5 lapse rate

Pair *= 100
Pair

plt.plot(Pair, alt*1e-3)
plt.grid(True)
plt.xlabel('Pressure [pa]')
plt.ylabel('Altitude [km]')
plt.ylim(0, 15)
plt.savefig('fig4', dpi=200)

Tair = read[0]['T0']+273.15 -0.006666666666668182*alt[:int(12000/7.5)]
Tair_p = Tair[-1]*np.ones(len(alt[int(12000/7.5):]))
Tair = np.concatenate((Tair, Tair_p), axis=0)

plt.plot(Tair, alt*1e-3)
plt.grid(True)
plt.xlabel('Temperature [K]')
plt.ylabel('Altitude [km]')
plt.ylim(0, 15)
plt.savefig('fig5', dpi=200)

PairTair = pd.read_fwf('PairTair.txt', header=None)

PairTair

## Usando dados de Wyoming

In [ ]:
head = read[0]
date = datetime.strptime(head['datei'], '%d/%m/%Y').date()
station = 'SBMN'

PairTair = WyomingUpperAir.request_data(date, station)

In [ ]:
PairTair.head()

In [ ]:
PairTair

In [ ]:
Pair = 100*PairTair['pressure']
Tair = 273.15 + PairTair['temperature']

bg = np.mean(s355[-500:])  # O background eh a media dos ultimos 500 bins
n_alt = alt[11:3145]       # Preciso ajustar o tamanho dos arrays de altitude e sinal
                           # pra poder fazer a interpolacao
n_s355 = s355[11:3145]     # '''

x = PairTair['height']
y = Pair
f = interp1d(x, y, kind='cubic')
newPair = f(n_alt)


y = Tair
f = interp1d(x, y, kind='cubic')
newTair = f(n_alt)

In [ ]:
plt.plot(newPair, n_alt*1e-3)
plt.grid(True)
plt.xlabel('Pressure [pa]')
plt.ylabel('Altitude [km]')
plt.ylim(0, 20)
plt.savefig('Pres_profile', dpi=200)

In [ ]:
plt.plot(newTair, n_alt*1e-3)
plt.grid(True)
plt.xlabel('Temperature [K]')
plt.ylabel('Altitude [km]')
plt.ylim(0, 20)
plt.savefig('Temp_profile', dpi=200)

# Molecular Scattering

In [ ]:
%run alphabeta.ipynb

In [ ]:
lambd = 355*1e-9
co2 = 400


alpha_mol, beta_mol, LR_mol = alphabeta(newPair, newTair, lambd, co2)

In [ ]:
plt.plot(alpha_mol*1e6, n_alt*1e-3)
plt.grid(True)
plt.xlabel('alpha molecular [1/Mm]')
plt.ylabel('Altitude [km]')
plt.savefig('fig6', dpi=200)

In [ ]:
plt.plot(beta_mol*1e6, n_alt*1e-3)
plt.grid(True)
plt.xlabel('beta molecular [1/Mm 1/sr]')
plt.ylabel('Altitude [km]')
plt.savefig('fig7', dpi=200)

# Obtaining overlap

In [ ]:
overlap = pd.read_csv('overlap_narrow.csv', header=None)
overlap

In [ ]:
plt.plot(overlap[0])
plt.grid()

# built the simulated molecular signal

In [ ]:
s355 = s355 - bg          # Subtraindo o background
s355 = np.concatenate((s355[:2000]/overlap[0], s355[2000:]), axis=0)   #Dividindo pelo overlap
n_alt = alt[11:3145]
n_s355 = s355[11:3145]

In [ ]:
meteo_alt = n_alt

In [ ]:
model = (beta_mol * np.exp(-2*integrate.cumtrapz(alpha_mol, meteo_alt, initial=0))) / meteo_alt**2

In [ ]:
plt.semilogx(model*meteo_alt*meteo_alt, meteo_alt*1e-3)
plt.scatter((n_s355)*n_alt*n_alt, n_alt*1e-3, s=0.5, c='orange')
plt.grid(True)
plt.ylabel('Altitude [km]')
plt.xlim(1e-10, 1e15)

## Determinando a região do ajuste

In [ ]:
alt_min = 8.   # km
alt_max = 15   # km

index_min = int(alt_min*1e3/7.5)
index_max = int(alt_max*1e3/7.5)

In [ ]:
plt.scatter(model[index_min:index_max], s355[index_min:index_max], facecolors = 'none', edgecolors = 'orange')
plt.grid(True)
plt.xlabel('Model molecular')
plt.ylabel('Measured signal')

In [ ]:
f = lambda x, a, b: a*x + b

fit = optimize.curve_fit(f, model[index_min:index_max], n_s355[index_min:index_max], p0=(1e13, 116))

a = fit[0][0]
b = fit[0][1]

fit

In [ ]:
plt.scatter(model[index_min:index_max], n_s355[index_min:index_max], facecolors = 'none', edgecolors = 'orange')
plt.plot(model[index_min:index_max], f(model[index_min:index_max], a, b))
plt.grid(True)
plt.xlabel('Model molecular')
plt.ylabel('Measured signal')

In [ ]:
plt.semilogx(model*meteo_alt*meteo_alt, meteo_alt*1e-3)
plt.scatter((n_s355)*n_alt*n_alt, n_alt*1e-3, c='orange', s=0.5)
plt.semilogx(model*a*meteo_alt*meteo_alt, meteo_alt*1e-3, c='green')
plt.grid(True)
plt.ylabel('Altitude [km]')
plt.xlim(1e-10, 1e15)
plt.savefig('fig8', dpi=200)

In [ ]:
%run fernald_new1.ipynb

# Devemos sempre entrar com vetores coluna no algoritmo de inversao

zref = np.array([6*1e3, 14*1e3])
LR = 28.

z = n_alt[..., None]
pr = n_s355[..., None]
ref = zref[..., None]
lambd = 355*1e-9
LidarRatio = LR
Pair = newPair[..., None]
Tair = newTair[..., None]


beta_par, alpha_par, LR_par, model_mol, ab_fit = fernald_new1(z,pr,ref,lambd,LidarRatio,Pair,Tair)

## Devemos restringir o plot a partir de 750m porque antes o overlap varia mto

In [ ]:
lim = 100
beta_par['aer']  = beta_par['aer'][lim:]
beta_par['mol']  = beta_par['mol'][lim:]
beta_par['tot']  = beta_par['tot'][lim:]
alpha_par['aer'] = alpha_par['aer'][lim:]
alpha_par['mol'] = alpha_par['mol'][lim:]
alpha_par['tot'] = alpha_par['tot'][lim:]
n_alt = n_alt[lim:]

In [ ]:
plt.plot(beta_par['aer']*1e6, n_alt*1e-3)
plt.plot(beta_par['mol']*1e6, n_alt*1e-3)
plt.plot(beta_par['tot']*1e6, n_alt*1e-3)
plt.grid(True)
plt.legend(['aer', 'mol', 'tot'])
# plt.xlim(-5, 14)
plt.ylim(0, 10)
plt.xlabel('beta [1/Mm sr]')
plt.ylabel('Altitude [km]')
plt.savefig('backscatter', dpi=200)

In [ ]:
plt.plot(alpha_par['aer']*1e6, n_alt*1e-3)
plt.plot(alpha_par['mol']*1e6, n_alt*1e-3)
plt.plot(alpha_par['tot']*1e6, n_alt*1e-3)
plt.grid(True)
plt.legend(['aer', 'mol', 'tot'])
# plt.xlim(-5, 14)
plt.ylim(0, 10)
plt.xlabel('beta [1/Mm]')
plt.ylabel('Altitude [km]')
plt.savefig('alpha', dpi=200)